In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210206-1439-0.2.61-3c86d3ba497a.log


In [2]:
recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}
mt1 = hl.import_vcf('gs://pgr-wes-r20/GL_by_chrom/CNCD_Freeze_Two.15.GL.vcf.gz', reference_genome='GRCh38', contig_recoding=recode, force_bgz=True, array_elements_required=False)
ds_result = hl.filter_intervals(mt1, [hl.parse_locus_interval('chr15:20531856-20541800', reference_genome='GRCh38')])
print('1')
participant_ids = ['CNCD_QNF329_QNF329']

ds_result = ds_result.drop('PL')

ds_result.rows().show()
print(ds_result.count())
ds_result1 = ds_result.filter_cols(hl.literal(participant_ids).contains(ds_result.s))
print(ds_result1.count())
ds_result1.rows().show()

print('2')
ds_result1 = hl.split_multi_hts(ds_result1, permit_shuffle=True)
ds_result1.rows().show()

print('3')
print('Annotate Heterozygous Alleles')
ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
ds_result1.count()
print('Annotate N Hets')
ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
ds_result4.count()
print('Annotate Homozygous Alleles')
ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
ds_result3.count()
print('Annotating N Homs')
ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

ds_result5.rows().show()

data3 = ds_result5.annotate_rows(
    info=ds_result5.info.annotate(hets=ds_result5.hets,
                             n_het=ds_result5.n_het,
                             homs=ds_result5.homs,
                             n_hom=ds_result5.n_hom)
                                )

hl.export_vcf(data3, "gs://test1haseeb/GOLGA6L6_20K.vcf")

1


2021-02-06 14:40:12 Hail: INFO: Coerced sorted dataset
2021-02-06 14:40:12 Hail: INFO: reading 1 of 55 data partitions


+----------------+---------------+-----------------------------------+
| locus          | alleles       | rsid                              |
+----------------+---------------+-----------------------------------+
| locus<GRCh38>  | array<str>    | str                               |
+----------------+---------------+-----------------------------------+
| chr15:20534182 | ["C","A"]     | "15_20534182_C_A"                 |
| chr15:20534186 | ["C","T"]     | "15_20534186_C_T"                 |
| chr15:20534194 | ["C","T"]     | "15_20534194_C_T"                 |
| chr15:20534198 | ["C","G","A"] | "15_20534198_C_G;15_20534198_C_A" |
| chr15:20534199 | ["C","A"]     | "15_20534199_C_A"                 |
| chr15:20534207 | ["C","T"]     | "15_20534207_C_T"                 |
| chr15:20534209 | ["G","A"]     | "15_20534209_G_A"                 |
| chr15:20534219 | ["C","T"]     | "15_20534219_C_T"                 |
| chr15:20534226 | ["C","T"]     | "15_20534226_C_T"                 |
| chr15:20534237 | ["C","T"]     | "15_20534237_C_T"                 |
+----------------+---------------+-----------------------------------+

+----------+----------+---------------------+--------------+
|     qual | filters  | info.AF             | info.AQ      |
+----------+----------+---------------------+--------------+
|  float64 | set<str> | array<float64>      | array<int32> |
+----------+----------+---------------------+--------------+
| 5.90e+01 | NA       | [2.60e-05]          | [59]         |
| 6.80e+01 | NA       | [5.10e-05]          | [68]         |
| 8.10e+01 | NA       | [2.60e-05]          | [81]         |
| 9.60e+01 | NA       | [5.10e-05,2.60e-05] | [96,39]      |
| 5.90e+01 | NA       | [2.60e-05]          | [59]         |
| 5.30e+01 | NA       | [5.10e-05]          | [53]         |
| 6.70e+01 | NA       | [5.10e-05]          | [67]         |
| 7.80e+01 | NA       | [2.60e-05]          | [78]         |
| 6.10e+01 | NA       | [2.60e-05]          | [61]         |
| 2.22e+02 | NA       | [1.54e-04]          | [222]        |
+----------+----------+---------------------+--------------+
showing top 10 rows

2021-02-06 14:40:27 Hail: INFO: Coerced sorted dataset
2021-02-06 14:40:28 Hail: INFO: reading 1 of 55 data partitions


(372, 19509)


...

2021-02-06 14:43:30 Hail: INFO: Coerced sorted dataset
2021-02-06 14:43:30 Hail: INFO: reading 1 of 55 data partitions
2021-02-06 14:43:42 Hail: INFO: Coerced sorted dataset
2021-02-06 14:43:42 Hail: INFO: reading 1 of 55 data partitions


Annotating N Homs


2021-02-06 14:44:02 Hail: INFO: Coerced sorted dataset
2021-02-06 14:44:02 Hail: INFO: reading 1 of 55 data partitions
2021-02-06 14:44:28 Hail: INFO: Coerced sorted dataset
2021-02-06 14:44:41 Hail: INFO: Coerced sorted dataset
2021-02-06 14:44:41 Hail: INFO: reading 1 of 55 data partitions
2021-02-06 14:45:01 Hail: INFO: Ordering unsorted dataset with network shuffle


+----------------+------------+-----------------------------------+----------+
| locus          | alleles    | rsid                              |     qual |
+----------------+------------+-----------------------------------+----------+
| locus<GRCh38>  | array<str> | str                               |  float64 |
+----------------+------------+-----------------------------------+----------+
| chr15:20534182 | ["C","A"]  | "15_20534182_C_A"                 | 5.90e+01 |
| chr15:20534186 | ["C","T"]  | "15_20534186_C_T"                 | 6.80e+01 |
| chr15:20534194 | ["C","T"]  | "15_20534194_C_T"                 | 8.10e+01 |
| chr15:20534198 | ["C","A"]  | "15_20534198_C_G;15_20534198_C_A" | 9.60e+01 |
| chr15:20534198 | ["C","G"]  | "15_20534198_C_G;15_20534198_C_A" | 9.60e+01 |
| chr15:20534199 | ["C","A"]  | "15_20534199_C_A"                 | 5.90e+01 |
| chr15:20534207 | ["C","T"]  | "15_20534207_C_T"                 | 5.30e+01 |
| chr15:20534209 | ["G","A"]  | "15_20534209_G_A"                 | 6.70e+01 |
| chr15:20534219 | ["C","T"]  | "15_20534219_C_T"                 | 7.80e+01 |
| chr15:20534226 | ["C","T"]  | "15_20534226_C_T"                 | 6.10e+01 |
+----------------+------------+-----------------------------------+----------+

+----------+---------------------+--------------+---------+-----------+
| filters  | info.AF             | info.AQ      | a_index | was_split |
+----------+---------------------+--------------+---------+-----------+
| set<str> | array<float64>      | array<int32> |   int32 |      bool |
+----------+---------------------+--------------+---------+-----------+
| NA       | [2.60e-05]          | [59]         |       1 |     false |
| NA       | [5.10e-05]          | [68]         |       1 |     false |
| NA       | [2.60e-05]          | [81]         |       1 |     false |
| NA       | [5.10e-05,2.60e-05] | [96,39]      |       2 |      true |
| NA       | [5.10e-05,2.60e-05] | [96,39]      |       1 |      true |
| NA       | [2.60e-05]          | [59]         |       1 |     false |
| NA       | [5.10e-05]          | [53]         |       1 |     false |
| NA       | [5.10e-05]          | [67]         |       1 |     false |
| NA       | [2.60e-05]          | [78]         |       1 |     false |
| NA       | [2.60e-05]          | [61]         |       1 |     false |
+----------+---------------------+--------------+---------+-----------+

+------------+-------+------------+-------+
| hets       | n_het | homs       | n_hom |
+------------+-------+------------+-------+
| array<str> | int64 | array<str> | int64 |
+------------+-------+------------+-------+
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
| []         |     0 | []         |     0 |
+------------+-------+------------+-------+
showing top 10 rows

2021-02-06 14:45:28 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
    'hets' (row)
    'n_het' (row)
    'homs' (row)
    'n_hom' (row)
2021-02-06 14:45:40 Hail: INFO: Coerced sorted dataset
2021-02-06 14:45:40 Hail: INFO: reading 1 of 55 data partitions
2021-02-06 14:46:46 Hail: INFO: Coerced sorted dataset
2021-02-06 14:47:23 Hail: INFO: Coerced sorted dataset
2021-02-06 14:47:23 Hail: INFO: reading 1 of 55 data partitions
2021-02-06 14:48:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:50:44 Hail: INFO: merging 2 files totalling 274.2K...
2021-02-06 14:50:44 Hail: INFO: while writing:
    gs://test1haseeb/GOLGA6L6_20K.vcf
  merge time: 432.851ms
